In [3]:
import json
import cv2
import numpy as np
import pandas as pd
from typing import Dict, List, Tuple

In [5]:
# Ustawienia
JSON_PATH = "dataset/label.json"  # przykładowa ścieżka
VIDEO_PATH = "dataset/traffic.mp4"
OUTPUT_VIDEO = "dataset/result.mp4"
OUTPUT_CSV = "dataset/results.csv"
LINE_Y = 300  # linia, którą obiekty mają przecinać

In [7]:
# Tracker z korektą centroidu i TTL
class BottomCenterTracker:
    def __init__(self, max_distance: int = 80, ttl: int = 20):
        self.next_id = 0
        self.objects = {}
        self.ttl_map = {}
        self.crossed = set()
        self.max_distance = max_distance
        self.ttl = ttl

    def update(self, detections: List[Tuple[str, int, int, int, int]]) -> List[Tuple[int, str, int, int, int, int, bool]]:
        updated = []
        bottom_centers = [((x1 + x2) // 2, y2) for _, x1, y1, x2, y2 in detections]
        matched = {}
        used_ids = set()

        for i, (cls, x1, y1, x2, y2) in enumerate(detections):
            cx, cy = bottom_centers[i]
            best_match = None
            best_dist = float('inf')
            for obj_id, (px, py) in self.objects.items():
                if obj_id in used_ids:
                    continue
                dist = np.hypot(cx - px, cy - py)
                if dist < self.max_distance and dist < best_dist:
                    best_match = obj_id
                    best_dist = dist
            if best_match is not None:
                obj_id = best_match
                self.objects[obj_id] = (cx, cy)
                self.ttl_map[obj_id] = 0
                matched[i] = obj_id
                used_ids.add(obj_id)
            else:
                obj_id = self.next_id
                self.objects[obj_id] = (cx, cy)
                self.ttl_map[obj_id] = 0
                matched[i] = obj_id
                self.next_id += 1
                used_ids.add(obj_id)

        for obj_id in list(self.ttl_map.keys()):
            self.ttl_map[obj_id] += 1
            if self.ttl_map[obj_id] > self.ttl:
                self.objects.pop(obj_id, None)
                self.ttl_map.pop(obj_id, None)

        for i, (cls, x1, y1, x2, y2) in enumerate(detections):
            obj_id = matched[i]
            _, cy = bottom_centers[i]
            crossed = False
            if cy >= LINE_Y and obj_id not in self.crossed:
                crossed = True
                self.crossed.add(obj_id)
            updated.append((obj_id, cls, x1, y1, x2, y2, crossed))

        return updated

In [9]:
# Interpolacja z przesunięciem bboxów
def interpolate_annotations_adjusted(json_path: str, width: int, height: int) -> Dict[int, List[Tuple[str, int, int, int, int]]]:
    with open(json_path, "r") as f:
        data = json.load(f)

    annotations = {}

    for item in data:
        for ann in item.get("annotations", []):
            for result in ann.get("result", []):
                label = result.get("value", {}).get("labels", ["car"])[0]
                sequence = result.get("value", {}).get("sequence", [])
                prev_frame = None
                prev_box = None

                for idx, entry in enumerate(sequence):
                    if not entry.get("enabled", True):
                        continue
                    frame = entry["frame"]
                    x = entry["x"] / 100.0 * width
                    y = entry["y"] / 100.0 * height
                    w = entry["width"] / 100.0 * width
                    h = entry["height"] / 100.0 * height

                    # Korekta pozycji - cofnięcie bboxa lekko w tył
                    x -= 0.1 * w
                    y -= 0.05 * h

                    x1, y1 = int(round(x)), int(round(y))
                    x2, y2 = int(round(x + w)), int(round(y + h))
                    box = (label, x1, y1, x2, y2)

                    if prev_frame is not None and frame > prev_frame + 1:
                        for f in range(prev_frame + 1, frame):
                            alpha = (f - prev_frame) / (frame - prev_frame)
                            interp_box = tuple(
                                int(prev_box[i] * (1 - alpha) + box[i] * alpha)
                                if i > 0 else label
                                for i in range(5)
                            )
                            annotations.setdefault(f, []).append(interp_box)

                    annotations.setdefault(frame, []).append(box)
                    prev_frame = frame
                    prev_box = box

    return annotations

In [11]:
# Główna funkcja
def main():
    cap = cv2.VideoCapture(VIDEO_PATH)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    annotations = interpolate_annotations_adjusted(JSON_PATH, width, height)
    tracker = BottomCenterTracker()

    out = cv2.VideoWriter(OUTPUT_VIDEO, cv2.VideoWriter_fourcc(*"mp4v"), fps, (width, height))
    results = []
    frame_idx = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        detections = annotations.get(frame_idx, [])
        tracked = tracker.update(detections)

        for obj_id, cls, x1, y1, x2, y2, crossed in tracked:
            color = (0, 255, 0) if not crossed else (0, 0, 255)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, f"{cls}-{obj_id}", (x1, y1 - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            results.append((frame_idx, obj_id, cls, x1, y1, x2, y2, crossed))

        cv2.line(frame, (0, LINE_Y), (width, LINE_Y), (255, 0, 0), 2)
        cv2.putText(frame, f"Przekroczylo linie: {len(tracker.crossed)}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

        out.write(frame)
        frame_idx += 1

    cap.release()
    out.release()

    df = pd.DataFrame(results, columns=["frame", "id", "class", "x1", "y1", "x2", "y2", "crossed"])
    df.to_csv(OUTPUT_CSV, index=False)

main()